# Deep Learning and Tensorflow Fundamentals

In this notebook, we're going to cover some of the most fundamental concepts of tensors using Tensorflow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors and NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself

## Introduction to Tensors

In [1]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.9.1


In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [5]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [6]:
# Check the dimension of our vector
vector.ndim

1

In [7]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7],
                      [7, 10]])

matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [8]:
# Check dimension of the matrix
matrix.ndim

2

In [9]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8.,9.]], dtype = tf.float16) # specify the datatype

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [10]:
another_matrix.ndim, another_matrix.dtype

(2, tf.float16)

In [12]:
# Lets create a tensor

tensor = tf.constant([
    [[1,2,3],[4,5,6]],
    [[7,8,9],[10,11,12]],
    [[13,14,15], [16,17,18]]
])
tensor, tensor.ndim, tensor.shape

(<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
 array([[[ 1,  2,  3],
         [ 4,  5,  6]],
 
        [[ 7,  8,  9],
         [10, 11, 12]],
 
        [[13, 14, 15],
         [16, 17, 18]]])>,
 3,
 TensorShape([3, 2, 3]))

What we've created so far:

* Scalar: a single number
* Vector: a number with direction (eg wind speed and direction)
* Matrix: a 2-dimensional array of numbers
* Tensor: an n-dimensional array of numbers (when n can be any number, a 0 dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

## Creating Tensors with tf.Variable

In [14]:
# Create the same tensor as we created above with tf.Variable()
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [16]:
# Lets try changing one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [17]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [18]:
# Lets try changing our unchangeable tensor
unchangeable_tensor[0] = 7

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [19]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

**Note:** Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors as Tensorflow does this for you. However, if in doubt, use tf.constant and change it later if needed

## Creating Random Tensors

Random tensors are tensors of some arbitrary size which contain random numbers

In [20]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape = (3,2))
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193765, -1.8107855 ]], dtype=float32)>

In [22]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3, 2))
random_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193765, -1.8107855 ]], dtype=float32)>

In [23]:
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

## Shuffle the order of elements in a tensor

In [24]:
# Shuffle a tensor (valuable for when you want to shuffle the data so that the inherent order doesn't affect learning)
not_shuffle = tf.constant([
    [10,7],
    [3,4],
    [2,5]
])
not_shuffle.ndim

2

In [25]:
# Shuffle our non-shuffled tensor
shuffled_tensor = tf.random.shuffle(not_shuffle, seed = 42)
shuffled_tensor

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]])>

In [27]:
shuffled_tensor_2 = tf.random.shuffle(not_shuffle, seed = 42)
shuffled_tensor_2

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

The order is different for the same seed! The better method is the following:

In [34]:
tf.random.set_seed(42) #global level random seed
shuffled_tensor_3 = tf.random.shuffle(not_shuffle, seed = 42) # operation level random seed
shuffled_tensor_3

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

It looks like if we want our shuffled tensors in the same order, we've got to use the global level random seed and the operation level random seed

> Rule 4: "If both the global and the operation seed are set, both seeds are used in conjugation to determine the random sequence"

## Other ways to make tensors

In [2]:
# Create a tensor of all ones
tf.ones(shape = (3,2), dtype=tf.int16)

<tf.Tensor: shape=(3, 2), dtype=int16, numpy=
array([[1, 1],
       [1, 1],
       [1, 1]], dtype=int16)>

In [3]:
# Create a tensor of all zeros
tf.zeros(shape = (3,2), dtype = tf.int16)

<tf.Tensor: shape=(3, 2), dtype=int16, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int16)>

## Creating tensors from NumPy arrays

The main difference between NumPy arrays and Tensorflow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [4]:
# You can also turn NumPy arrays into tensors

import numpy as np
numpy_A = np.arange(1, 25, dtype = np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [8]:
A = tf.constant(numpy_A, shape = (2,3,4))
B = tf.constant(numpy_A)
A,B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

## Getting information from tensors

* Shape
* Rank
* Axis or dimension
* Size

|Attribute|Meaning|Code|
|----|----|----|
|Shape|The length (number of elements) of each of the dimensions of a tensor|`tensor.shape`|
|Rank|The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix is rank 2, a tensor has rank n|`tensor.ndim`|
|Axis or dimension|A particular dimension of a tensor|`tensor[0], tensor[:,1]...`|
|Size|The total number of items in the tensor|`tf.size(tensor)`|

In [9]:
# Create a rank 4 tensor (4 dimensions)

rank_4_tensor = tf.zeros(shape = (2,3,4,5), dtype = tf.int16)

In [10]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=int16, numpy=
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]], dtype=int16)>

In [11]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [15]:
# Get various attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of the tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor))

Datatype of every element:  <dtype: 'int16'>
Number of dimensions (rank):  4
Shape of the tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)


In [16]:
# Get various attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of the tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'int16'>
Number of dimensions (rank):  4
Shape of the tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor: 120


## Indexing and expanding tensors

Tensors can be indexed just like Python lists

In [21]:
# Get the first 2 elements of each dimension

rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int16, numpy=
array([[[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]]], dtype=int16)>

In [23]:
# Get the first element from each dimension except the last one
rank_4_tensor[:1,:1,:1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=int16, numpy=array([[[[0, 0, 0, 0, 0]]]], dtype=int16)>

In [24]:
# Create a rank 2 tensor (2 dimensions)

rank_2_tensor = tf.constant(np.array([10,7,7,10]).reshape(2,2))
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [25]:
# Get the last item of each row of our rank 2 tensor

rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>

In [26]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # equivalent to rank_2_tensor[:,:,tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 7],
        [10]]])>

In [27]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 7],
        [10]]])>

In [28]:
tf.expand_dims(rank_2_tensor, axis = 0) # expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 7, 10]]])>

## Manipulating Tensors (tensor operations)

### **Basic operations**

### `+`,`-`,`*`,`/`

In [29]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10,7],[3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [30]:
# Multiplication also works

tensor*10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

In [31]:
# Subtraction if you want
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]])>

In [32]:
# We can use the tensorflow built-in functions to 
tf.math.multiply(tensor, 10) # or tf.multiply
# tf.add and so on

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]])>

### **Matrix Multiplication**

In machine learning, matrix multiplication is one of the most important operations.

In [33]:
tf.matmul(tensor,tensor)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [34]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]])>

In [36]:
X = tf.random.normal((3,2))
Y = tf.random.normal((3,2))

tf.matmul(X,Y)

InvalidArgumentError: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [37]:
reshape_Y = tf.reshape(Y, (2, 3))
tf.matmul(X, reshape_Y)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 2.6234064 , -0.03282936,  0.9239514 ],
       [ 0.54325426,  0.19752294, -0.6149682 ],
       [-0.6397762 , -0.5050194 ,  1.7991936 ]], dtype=float32)>

In [39]:
Y_transpose = tf.transpose(Y)
tf.matmul(X, Y_transpose)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.42693493,  2.956439  ,  0.5502796 ],
       [ 0.03460938, -0.14429766,  0.15648429],
       [ 0.03096806,  1.1785257 , -0.24099195]], dtype=float32)>

### **The dot product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`

In [42]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes = 1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.7864301 ,  0.42229247],
       [-0.5155717 ,  0.8280754 ]], dtype=float32)>

In [43]:
# Check the values of Y, reshape Y and transposed Y
Y, tf.transpose(Y), tf.reshape(Y, (2, 3))

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.09870622,  0.315094  ],
        [-1.2733246 ,  2.0498881 ],
        [ 0.04463004,  0.4446101 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[-0.09870622, -1.2733246 ,  0.04463004],
        [ 0.315094  ,  2.0498881 ,  0.4446101 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[-0.09870622,  0.315094  , -1.2733246 ],
        [ 2.0498881 ,  0.04463004,  0.4446101 ]], dtype=float32)>)

Generally transpose is used rather than reshape to get the dimensions in order for multiplication

## Changing the datatype of a tensor

In [45]:
# Create a new tensor with default datatype (float32)

B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [46]:
A = tf.constant([1,2])
A.dtype

tf.int32

In [47]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype = tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [48]:
# Change from int32 to float32
E = tf.cast(A, dtype = tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>,
 tf.float32)

## Aggregating Tensors

Condensing tensors from multiple values down to a smaller number of values

In [49]:
# Get the absolute values
D = tf.constant([-7,-10])
D, tf.abs(D)

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>)

Lets go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [50]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size = 50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([27, 37, 99, 41, 59, 21, 38, 60, 72,  7, 36, 91, 52, 84, 15, 28, 63,
       48, 16, 37, 62, 54,  6, 81, 51, 18, 65, 73, 56, 25, 77, 41, 14, 16,
       31, 34, 71,  0, 16, 48, 95, 94,  5, 36, 84,  8, 66, 37, 34, 74])>

In [51]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [52]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [53]:
# Find the minimum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int32, numpy=99>

In [54]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int32, numpy=46>

In [55]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int32, numpy=2303>

In [58]:
# Find the variance
tf.math.reduce_variance(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=719.65643>

In [60]:
# Find the standard deviation
np.sqrt(tf.math.reduce_variance(tf.cast(E, dtype = tf.float32)).numpy()), tf.math.reduce_std(tf.cast(E, dtype = tf.float32))

(26.826412, <tf.Tensor: shape=(), dtype=float32, numpy=26.826412>)

In [62]:
## Find the positional maximum and minimum
tf.argmax(E), tf.argmin(E)

(<tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=37>)

In [63]:
E[tf.argmax(E)] == tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [64]:
# Find the minimum using the positional minimum index
E[tf.argmin(E)]

<tf.Tensor: shape=(), dtype=int32, numpy=0>

## Squeezing a tensor (removing all 1-dimension axes)

In [65]:
# Create a tensor to get started
G = tf.constant(tf.random.uniform(shape = [50]), shape = (1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[2.1037507e-01, 5.3716254e-01, 9.4057024e-01, 5.2221382e-01,
           2.4023807e-01, 1.2451923e-01, 4.8366606e-01, 1.0977030e-02,
           7.5326490e-01, 5.4109669e-01, 3.3204007e-01, 1.7899394e-01,
           1.5802419e-01, 8.2042217e-03, 1.3539743e-01, 6.6665471e-01,
           4.5777249e-01, 7.7848601e-01, 4.0838540e-01, 9.7474384e-01,
           2.1433830e-04, 4.0154016e-01, 4.8814261e-01, 5.8320713e-01,
           5.5715013e-01, 7.8261590e-01, 1.1955321e-01, 2.9103506e-01,
           1.7058372e-01, 8.0888987e-01, 6.6274643e-02, 6.5228534e-01,
           8.6421597e-01, 2.8068590e-01, 5.8367467e-01, 2.0047784e-02,
           9.8973393e-02, 1.4422393e-01, 6.8331575e-01, 1.2115216e-01,
           9.0466213e-01, 1.7279148e-02, 5.0847328e-01, 2.2163701e-01,
           6.0995793e-01, 6.9865465e-02, 1.7529094e-01, 7.8364265e-01,
           4.8960841e-01, 2.9545486e-01]]]]], dtype=float32)>

In [68]:
G.shape, tf.squeeze(G).shape

(TensorShape([1, 1, 1, 1, 50]), TensorShape([50]))

## One-Hot Encoding Tensors

In [80]:
# Create a list of indices
some_list = [0,1,2,3] # could be red, green, blue or purple

# One-hot encode our list of indices
tf.one_hot(some_list, depth=tf.convert_to_tensor(tf.unique(some_list)).shape[1])

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [81]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth=tf.convert_to_tensor(tf.unique(some_list)).shape[1], on_value="DL", off_value = "Human")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'DL', b'Human', b'Human', b'Human'],
       [b'Human', b'DL', b'Human', b'Human'],
       [b'Human', b'Human', b'DL', b'Human'],
       [b'Human', b'Human', b'Human', b'DL']], dtype=object)>

## Squaring, log, square root

In [82]:
# Create a new tensor

H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [83]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [84]:
# Find the squareroot (will error, method requires float)
tf.sqrt(tf.cast(H, dtype = tf.float16))

<tf.Tensor: shape=(9,), dtype=float16, numpy=
array([1.   , 1.414, 1.732, 2.   , 2.236, 2.45 , 2.646, 2.828, 3.   ],
      dtype=float16)>

In [86]:
# Find the log
tf.math.log(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

## Tensors and NumPy

Tensorflow interacts beautifully with NumPy arrays

In [87]:
# Create a tensor directly from a NumPy array

J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [88]:
# convert tensor back to numpy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [89]:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [90]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

## Finding access to GPUs

In [93]:
tf.config.list_physical_devices(), tf.config.list_physical_devices("GPU")

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')],
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

In [94]:
!nvidia-smi

Sat Jun 18 17:59:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.96       Driver Version: 512.96       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   55C    P8     8W /  N/A |   4438MiB /  6144MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------